In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
import keras

In [2]:


#Set the path to your train dataset directory
dataset_path = './dataset/train'

# Define the image size for resizing
image_size = (32, 32)

# Load and preprocess the dataset
X_train = []
y_train = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_train.append(image)
                y_train.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

classes = np.unique(y_train)

#Set the path to your test dataset directory
dataset_path = './dataset/test'

# Load and preprocess the dataset
X_test = []
y_test = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_test.append(image)
                y_test.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Perform train-test split: This has already been done in the dataset
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#Set the path to your Validation dataset directory
dataset_path = './dataset/val'

# Load and preprocess the dataset
X_val = []
y_val = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_val.append(image)
                y_val.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)


# Encode the shape labels into numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert the numerical labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))


Epoch 1/10
235/235 [==============================] - 9s 31ms/step - loss: 3.6263 - accuracy: 0.2293 - val_loss: 1.6087 - val_accuracy: 0.1948
Epoch 2/10
235/235 [==============================] - 8s 35ms/step - loss: 1.5749 - accuracy: 0.2400 - val_loss: 1.5774 - val_accuracy: 0.2824
Epoch 3/10
235/235 [==============================] - 8s 32ms/step - loss: 1.4264 - accuracy: 0.3485 - val_loss: 1.3458 - val_accuracy: 0.4052
Epoch 4/10
235/235 [==============================] - 8s 35ms/step - loss: 1.1128 - accuracy: 0.5393 - val_loss: 0.8769 - val_accuracy: 0.6700
Epoch 5/10
235/235 [==============================] - 8s 36ms/step - loss: 0.7544 - accuracy: 0.6924 - val_loss: 0.7113 - val_accuracy: 0.7296
Epoch 6/10
235/235 [==============================] - 7s 32ms/step - loss: 0.4515 - accuracy: 0.8309 - val_loss: 0.3258 - val_accuracy: 0.8876
Epoch 7/10
235/235 [==============================] - 8s 34ms/step - loss: 0.1963 - accuracy: 0.9279 - val_loss: 0.2436 - val_accuracy: 0.9176

# Check Predicted Shape

In [3]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 252ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 43ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 49ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 40ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 42ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 39ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 47ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 48ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 42ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 45ms/step
Predicted label: ['triangle']
Predicted label: ['circle']
Predicted label: ['trapezoid']
Predicted label: ['trapezoid']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted label: 

In [4]:
print(classes)

['circle' 'parallelogram' 'rectangle' 'trapezoid' 'triangle']


In [5]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 15ms/step - loss: 0.1748 - accuracy: 0.9480
Accuracy: 0.9480000138282776


# Training Model Using A tanh Activation In The Hidden Layer

In [6]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

Epoch 1/10
235/235 [==============================] - 11s 41ms/step - loss: 1.6569 - accuracy: 0.2288 - val_loss: 1.5476 - val_accuracy: 0.2732
Epoch 2/10
235/235 [==============================] - 10s 43ms/step - loss: 1.6066 - accuracy: 0.2075 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 3/10
235/235 [==============================] - 9s 40ms/step - loss: 1.6096 - accuracy: 0.1979 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 4/10
235/235 [==============================] - 7s 29ms/step - loss: 1.6096 - accuracy: 0.1980 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 5/10
235/235 [==============================] - 7s 31ms/step - loss: 1.6096 - accuracy: 0.1896 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 6/10
235/235 [==============================] - 7s 30ms/step - loss: 1.6096 - accuracy: 0.1960 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 7/10
235/235 [==============================] - 7s 30ms/step - loss: 1.6096 - accuracy: 0.1873 - val_loss: 1.6094 - val_accuracy: 0.20

# Predict

In [7]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 126ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 34ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 29ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 33ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 25ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 28ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 30ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 42ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 27ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 35ms/step
Predicted label: ['circle']
Predicted label: ['circle']
Predicted label: ['circle']
Predicted label: ['circle']
Predicted label: ['circle']
Predicted label: ['circle']
Predicted label: ['circle']
Pr

In [8]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 9ms/step - loss: 1.6094 - accuracy: 0.2000
Accuracy: 0.20000000298023224


# Training Model Using A Sigmoid Activation In The Output Layer

In [9]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

Epoch 1/10
235/235 [==============================] - 9s 35ms/step - loss: 5.5017 - accuracy: 0.4899 - val_loss: 0.4938 - val_accuracy: 0.8164
Epoch 2/10
235/235 [==============================] - 8s 35ms/step - loss: 0.3205 - accuracy: 0.8849 - val_loss: 0.2486 - val_accuracy: 0.9144
Epoch 3/10
235/235 [==============================] - 8s 35ms/step - loss: 0.1468 - accuracy: 0.9476 - val_loss: 0.1700 - val_accuracy: 0.9460
Epoch 4/10
235/235 [==============================] - 8s 35ms/step - loss: 0.1142 - accuracy: 0.9552 - val_loss: 0.2020 - val_accuracy: 0.9320
Epoch 5/10
235/235 [==============================] - 7s 29ms/step - loss: 0.0826 - accuracy: 0.9688 - val_loss: 0.1391 - val_accuracy: 0.9556
Epoch 6/10
235/235 [==============================] - 10s 42ms/step - loss: 0.0572 - accuracy: 0.9784 - val_loss: 0.1268 - val_accuracy: 0.9556
Epoch 7/10
235/235 [==============================] - 9s 37ms/step - loss: 0.0497 - accuracy: 0.9816 - val_loss: 0.1311 - val_accuracy: 0.965

In [10]:
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 166ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 40ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 44ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 47ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 40ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 40ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 45ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 46ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 40ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 41ms/step
Predicted label: ['trapezoid']
Predicted label: ['circle']
Predicted label: ['trapezoid']
Predicted label: ['parallelogram']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted la

In [11]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 10ms/step - loss: 0.1574 - accuracy: 0.9676
Accuracy: 0.9675999879837036


# Initial Model Using ReLU in Input and Hidden Layer But Varied Epoch and Batch Size

The optimizer learning rate is changed.

In [12]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
#opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=32, batch_size=64, validation_data=(X_val, y_val_one_hot))

Epoch 1/32
118/118 [==============================] - 10s 69ms/step - loss: 6.5607 - accuracy: 0.4140 - val_loss: 0.6962 - val_accuracy: 0.7376
Epoch 2/32
118/118 [==============================] - 8s 67ms/step - loss: 0.4249 - accuracy: 0.8484 - val_loss: 0.3004 - val_accuracy: 0.8940
Epoch 3/32
118/118 [==============================] - 9s 73ms/step - loss: 0.1807 - accuracy: 0.9407 - val_loss: 0.2123 - val_accuracy: 0.9308
Epoch 4/32
118/118 [==============================] - 7s 55ms/step - loss: 0.1017 - accuracy: 0.9640 - val_loss: 0.1637 - val_accuracy: 0.9504
Epoch 5/32
118/118 [==============================] - 6s 52ms/step - loss: 0.0600 - accuracy: 0.9764 - val_loss: 0.1653 - val_accuracy: 0.9472
Epoch 6/32
118/118 [==============================] - 6s 51ms/step - loss: 0.0444 - accuracy: 0.9843 - val_loss: 0.1367 - val_accuracy: 0.9636
Epoch 7/32
118/118 [==============================] - 7s 55ms/step - loss: 0.0302 - accuracy: 0.9901 - val_loss: 0.1421 - val_accuracy: 0.960

# Predict

In [13]:
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 103ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 28ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 27ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 25ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 27ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 31ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 30ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 23ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 27ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 28ms/step
Predicted label: ['triangle']
Predicted label: ['circle']
Predicted label: ['trapezoid']
Predicted label: ['parallelogram']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted lab

In [14]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 8ms/step - loss: 0.1328 - accuracy: 0.9720
Accuracy: 0.972000002861023
